In [ ]:
using  SymPy
using  Plots; pyplot()
using  LinearAlgebra

In [ ]:
L = 1
H = 1
d = 0.1
l = 0.1
η = 1e-10

In [ ]:
E₁ = 2100e3
E₂ = 210e3
h = 0.005

In [ ]:
ht = 0.1

In [ ]:
ν₁ = 0.30
ν₂ = 0.30

In [ ]:
x,x₁,y,y₁ = symbols("x,x₁,y,y₁", real = true)

In [ ]:
@show λ₁ = E₁*ν₁/((1+ν₁)*(1-2*ν₁))
@show λ₂ = E₂*ν₂/((1+ν₂)*(1-2*ν₂))
@show μ₁ = E₁/(2*(1+ν₁))
@show μ₂ = E₂/(2*(1+ν₂))

### Piecewise function

In [ ]:
heaviside(x) = 0.5 * (sign(x) + 1)

interval(x, a, b) = heaviside(x-a) - heaviside(x-b)

### Gaussian Kernel Function

In [ ]:
wₕ(x,x₁) = (1/(sqrt(2*pi)*h))*exp(-(x-x₁)^2/(2*h^2))

In [ ]:
λx(x) = sqrt(λ₁-λ₂) * interval(x,-l/2,l/2) 
λy(y) = sqrt(λ₁-λ₂) * interval(y,-d/2,d/2) 

In [ ]:
μx(x) = sqrt(μ₁-μ₂) * interval(x,-l/2,l/2) 
μy(y) = sqrt(μ₁-μ₂) * interval(y,-d/2,d/2) 

In [ ]:
λ2D(x₁,y₁) = λx(x₁)*λy(y₁) + λ₂
μ2D(x₁,y₁) = μx(x₁)*μy(y₁) + μ₂

In [ ]:
λFx = λx(x₁)*wₕ(x,x₁)
λFy = λy(y₁)*wₕ(y,y₁)
μFx = μx(x₁)*wₕ(x,x₁)
μFy = μy(y₁)*wₕ(y,y₁)

In [ ]:
smoothλ = SymPy.integrate(λFx,(x₁,-L/2-ht,L/2+ht))*integrate(λFy,(y₁,-H/2-ht,H/2+ht)) + λ₂
smoothμ = SymPy.integrate(μFx,(x₁,-L/2-ht,L/2+ht))*integrate(μFy,(y₁,-H/2-ht,H/2+ht)) + μ₂

In [ ]:
xs = range(-L/2,L/2, length=100)
ys = range(-H/2,H/2, length=100)

plot1 = contourf(xs, ys, λ2D, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal, camera=(0,90))
plot2 = contourf(xs, ys, smoothλ, c=:diverging_bwr_40_95_c42_n256, axis = nothing, showaxis =false,aspect_ratio=:equal,camera=(0,90))
plot(plot1,plot2)

In [ ]:
using Gmsh:gmsh
using  GridapGmsh
using  Gridap
using  Gridap.Geometry
using  Gridap.TensorValues
using  Gridap.Fields
using  Gridap.CellData
using  Gridap.ReferenceFEs

In [ ]:
const lsp = 0.0060
hf = lsp/2.1
hd = 30*hf
Lₚ = 1.0
Hₚ = 1.0
lₚ = 0.1
hₚ = 0.1
rc1 = 0.10
cw = 0.006
cl = 0.10

gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)


gmsh.model.geo.addPoint(-Lₚ/2, -Hₚ/2, 0.0, hd, 1)
gmsh.model.geo.addPoint(Lₚ/2, -Hₚ/2, 0.0, hd, 2)
gmsh.model.geo.addPoint(Lₚ/2, Hₚ/2, 0.0, hd, 3)
gmsh.model.geo.addPoint(-Lₚ/2, Hₚ/2, 0.0, hd, 4)

gmsh.model.geo.addPoint(-Lₚ/2, 0.5*cw, 0.0, hf, 100)
gmsh.model.geo.addPoint(-Lₚ/2+cl, 0.5*cw, 0.0, hf, 101)
gmsh.model.geo.addPoint(-Lₚ/2+cl, -0.5*cw, 0.0, hf, 102)
gmsh.model.geo.addPoint(-Lₚ/2, -0.5*cw, 0.0, hf, 103)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 100, 100)
gmsh.model.geo.addLine(100, 101, 101)
gmsh.model.geo.addLine(101, 102, 102)
gmsh.model.geo.addLine(102, 103, 103)
gmsh.model.geo.addLine(103, 1, 4)

gmsh.model.geo.addCurveLoop([1,2,3,100,101,102,103,4],1)
gmsh.model.addPhysicalGroup(2, [1],1)

gmsh.model.geo.addPoint(-lₚ/2, -hₚ/2, 0.0, hd, 11)
gmsh.model.geo.addPoint(lₚ/2, -hₚ/2, 0.0, hd, 12)
gmsh.model.geo.addPoint(lₚ/2, hₚ/2, 0.0, hd, 13)
gmsh.model.geo.addPoint(-lₚ/2, hₚ/2, 0.0, hd, 14)

gmsh.model.geo.addLine(11, 12, 11)
gmsh.model.geo.addLine(12, 13, 12)
gmsh.model.geo.addLine(13, 14, 13)
gmsh.model.geo.addLine(14, 11, 14)

gmsh.model.geo.addCurveLoop([11,12,13,14],2)
gmsh.model.addPhysicalGroup(2, [2],2)
gmsh.model.geo.addPlaneSurface([2], 2)

gmsh.model.geo.addPlaneSurface([2,-1], 1)

gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [3],2)

gmsh.model.setPhysicalName(2, 1, "Matrix")
gmsh.model.setPhysicalName(2, 2, "InHom")

gmsh.model.setPhysicalName(1, 1, "DirichletBot")
gmsh.model.setPhysicalName(1, 2, "DirichletTop")


gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", hd)
gmsh.model.mesh.field.setNumber(11, "XMin", -0.5*Lₚ)
gmsh.model.mesh.field.setNumber(11, "XMax", 0.5*Lₚ)
gmsh.model.mesh.field.setNumber(11, "YMin", -rc1)
gmsh.model.mesh.field.setNumber(11, "YMax", rc1)
gmsh.model.mesh.field.setAsBackgroundMesh(11)

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)

gmsh.write("MatrixWithSquareInclusion.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("MatrixWithSquareInclusion.msh")
writevtk(model ,"MatrixWithSquareInclusion")

In [ ]:
using Gridap.Geometry
labels = get_face_labeling(model)
dimension = 2
mat_tags = get_face_tag(labels,dimension);

In [ ]:
const matrix_tag = get_tag_from_name(labels,"Matrix")
const InHom_tag = get_tag_from_name(labels,"InHom")

In [ ]:
const Gc_matrix = 2.7
const Gc_Inhom = 27

In [ ]:
function Gc(s_id,tag)
        if tag == matrix_tag          
            return  Gc_matrix *s_id
        elseif tag == InHom_tag        
            return Gc_Inhom *s_id
        end
end

In [ ]:
order = 1
degree = 2*order
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
P_vol = (1.0/2)*I4
P_dev = I4_sym  - P_vol

In [ ]:
p = get_cell_points(Ω)

In [ ]:
reffe = ReferenceFE(lagrangian,Float64,order)
V₁ = FESpace(model, reffe, conformity=:H1)

In [ ]:
λn = lambdify(smoothλ)
μn = lambdify(smoothμ)

λh22(p) = λn.(p[1],p[2])
μh22(p) = μn.(p[1],p[2])

In [ ]:
V₁₂ = MultiFieldFESpace([V₁,V₁])

In [ ]:
λ_h,μ_h = interpolate_everywhere([λh22,μh22],V₁₂)

In [ ]:
writevtk(Ω,"MaterialProperties",cellfields= ["λ"=>λ_h,"μ"=>μ_h])

In [ ]:
function σ_mod(ε,ε_in,s_in,λ,μ)
    σ_elas = λ*tr(ε)*one(ε) + 2*μ*ε
    σ = (s_in ^2+η)*σ_elas
    return  σ
end

In [ ]:
function ψPos(ε_in,λ,μ)
    σ_elas = λ*tr(ε_in)*one(ε_in) + 2*μ*ε_in
    σ_array = get_array(σ_elas)
    Egn_Vals = eigvals(σ_array)
    Egn_Vecs = eigvecs(σ_array)
    σ1 = Egn_Vals[1]
    σ2 = Egn_Vals[2]
    n1 = VectorValue(Egn_Vecs[:,1])
    n2 = VectorValue(Egn_Vecs[:,2])
 if (σ1)  >= 0
        if (σ2)  >= 0
            ψPlus = 0.5*(ε_in ⊙ (σ1*(n1 ⊗ n1) + σ2*(n2 ⊗ n2)))
        else
            ψPlus = 0.5*(ε_in ⊙ (σ1*(n1 ⊗ n1)))
        end
    elseif (σ1)  < 0
        if (σ2)  >= 0
            ψPlus = 0.5*(ε_in ⊙ (σ2*(n2 ⊗ n2)))
        else
            ψPlus = 0.0
        end
    end
    return ψPlus
end

In [ ]:
function  new_EnergyState(ψPlusPrev_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in  >=ψPlusPrev_in
        ψPlus_out =ψPlus_in
    else
        ψPlus_out=ψPlusPrev_in
    end
    true,ψPlus_out
end

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = Gridap.solve(op)
    return  qh
end

In [ ]:
sId = CellState(1.0,dΩ)
shId = project(sId,model,dΩ,order)

In [ ]:
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["DirichletBot","DirichletTop"],
          dirichlet_masks=[(true,true), (false,true)])

uh = zero(V0_Disp)

In [ ]:
LoadTagId = get_tag_from_name(labels,"DirichletTop")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
function  stepPhaseField(uh_in,ψPlusPrev_in)

        a_PF(s,ϕ) = ∫( (Gc∘(shId,mat_tags))*lsp*∇(ϕ)⋅ ∇(s) + 2*ψPlusPrev_in*s*ϕ + ((Gc∘(shId,mat_tags))/lsp)*s*ϕ )*dΩ
        b_PF(ϕ) = ∫( ((Gc∘(shId,mat_tags))/lsp)*ϕ )*dΩ  
        op_PF = AffineFEOperator(a_PF,b_PF,U_PF,V0_PF)
        sh_out = Gridap.solve(op_PF) 
    
    return sh_out  
end

In [ ]:
function   stepDisp(uh_in,sh_in,uApp)
        uApp1(x) = VectorValue(0.0,0.0)
        uApp2(x) = VectorValue(0.0,uApp)
        U_Disp = TrialFESpace(V0_Disp,[uApp1,uApp2])

        a_Disp(u,v) = ∫( (ε(v) ⊙ (σ_mod∘(ε(u),ε(uh_in),sh_in,λ_h,μ_h)) ) )*dΩ
        b_Disp(v) = 0.0
        op_Disp = AffineFEOperator(a_Disp,b_Disp,U_Disp,V0_Disp)
        uh_out = Gridap.solve(op_Disp)
    return uh_out
end

In [ ]:
uApp = 0
delu = 1.5e-4
uAppMax = 0.011
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
StressVal = Float64[]
push!(Load, 0.0)
push!(Displacement, 0.0)
push!(StressVal, 0.0)

sPrev = CellState(1.0,dΩ)
sh = project(sPrev,model,dΩ,order)
ψPlusPrev = CellState(0.0,dΩ)

while uApp .< uAppMax 
        
    uApp = uApp .+ delu
    count = count .+ 1
    
    if uApp >= 0.0065
        delu = 5e-6
    end
   
    print("\n Entering displacemtent step$count :", float(uApp))
    
   for inner = 1:innerMax
        
        ψhPlusPrev = project(ψPlusPrev,model,dΩ,order)
        
        RelErr = abs(sum(∫( (Gc∘(shId,mat_tags))*lsp*∇(sh)⋅ ∇(sh) + 2*ψhPlusPrev*sh*sh + ((Gc∘(shId,mat_tags))/lsp)*sh*sh)*dΩ-∫( ((Gc∘(shId,mat_tags))/lsp)*sh)*dΩ))/abs(sum(∫( ((Gc∘(shId,mat_tags))/lsp)*sh)*dΩ))
        print("\n error = ",float(RelErr))
        
        sh = stepPhaseField(uh,ψhPlusPrev) 
        uh = stepDisp(uh,sh,uApp)
        
        ψhPos_in = ψPos∘(ε(uh),λ_h,μ_h) 
        update_state!(new_EnergyState,ψPlusPrev,ψhPos_in)
        
        if RelErr < 1e-8
            break 
        end 
    end
    
    Node_Force = sum(∫( n_Γ_Load ⋅ (σ_mod∘(ε(uh),ε(uh),sh,λ_h,μ_h)) ) *dΓ_Load)
   
    push!(Load, Node_Force[2])
    push!(Displacement, uApp)
    if uApp <= 0.005
        if mod(count,4) == 0 
             writevtk(Ω,"results_PhaseFieldBeamWithNotch$count",cellfields=
            ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh)])
        end
    elseif uApp > 0.005
        if mod(count,4) == 0 
             writevtk(Ω,"results_PhaseFieldBeamWithNotch$count",cellfields=
            ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh)])
        end
    end
end 

In [ ]:
dispPlot = plot(Displacement,Load,label = nothing,xticks = 0:0.001:0.011)
xlabel!("Displacement (mm)")
ylabel!("Load (N)")

In [ ]:
savefig(dispPlot,"DisplacementPlot.pdf")

In [ ]:
using DelimitedFiles
Disp = writedlm("DispDataSingleLayerPlatewithSquareInclusion.csv",  Displacement, ',')
Force = writedlm("LoadDataSingleLayerPlatewithSquareInclusion.csv",  Load, ',')

In [ ]:
using Plots
using CSV
using DataFrames
using StatsPlots

In [ ]:
disp3Data = []
load3Data = []
disp1111 = CSV.read("DispDataSingleLayerPlatewithSquareInclusion.csv", DataFrame)
load1111 = CSV.read("LoadDataSingleLayerPlatewithSquareInclusion.csv", DataFrame)
disp2222 = CSV.read("MatrixWithSquareInclusionNew.csv", DataFrame)

push!(disp3Data,disp1111[!, 1])
push!(disp3Data,disp2222[!, 1])
push!(load3Data,load1111[!, 1])
push!(load3Data,disp2222[!, 2])

In [ ]:
plotf =[]
plotf = plot(disp3Data[1],load3Data[1]*1e-3,label = "Proposed model", lw = 2,legend=:topleft)
plotf = plot!(disp3Data[2],load3Data[2],label = "Yin, B. B., & Zhang, L. W. (2019)", lw = 2,legend=:topleft)
plotf
xlabel!("Displacement (mm)")
ylabel!("Load (KN)")

In [ ]:
savefig(plotf,"LoadDispRectangularWithSingleFiberInclusion.pdf")